<a href="https://colab.research.google.com/github/LuizWalker/PIM---Processamento-de-Imagens-Medicas/blob/main/Atividades_Parte_2/DICOM_Atividade_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1 - Abrir as imagens DICOM:

  #Ressonância
  #Tomografia
  #Radiografia

#2 - Identificar a diferença entre cada uma delas (cores, formato, dimensão, quantidade de slice).

#3 - Calcular o histograma das imagens e verificar os valores que cada uma tem

#4 - Verificar se é possível gerar o plot 3d

#5 - Como é possível fazer o janelamento das imagens

In [ ]:
# Importanto as bibliotecas novas #

!pip install pydicom
import pydicom 
from pydicom.data import get_testdata_file


# Bibliotecas já utilizada anteriormente #

import cv2 #Biblioteca OpenCV, utilizada para o processamento digital de imagens
from google.colab.patches import cv2_imshow # Função para o colab exibir imagens do OpenCV
import numpy as np
import matplotlib.pyplot as plt
import copy
from skimage.metrics import structural_similarity
from PIL import Image
from scipy import ndimage

In [ ]:
# Importanto as imagens do Github

